# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 ноября 2017, 23:59   
**Штраф за опоздание:** -2 балла после 23:59  4 декабря, -4 балла после 23:59 11 декабря, -6 баллов после 23:59 18 декабря

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

**Разбаловка:**   
За задание можно получить 10 баллов. Для этого нужно следующее:
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает    
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала качественнее, чем у дерева из sklearn

In [31]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [51]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print 'invalid criterion name'
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print 'invalid max_features name'
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        
        l_part = l_c / l_s
        r_part = r_c / r_s
        
        l = (1 - np.sum(l_part * l_part, axis = 1)).reshape(-1, 1)
        r = (1 - np.sum(r_part * r_part, axis = 1)).reshape(-1, 1)
        
        return (l * l_s + r * r_s)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        
        l_part = l_c / l_s
        r_part = r_c / r_s
        
        l = ( - np.sum(l_part * np.log(l_part), axis = 1)).reshape(-1, 1)
        r = ( - np.sum(r_part * np.log(r_part), axis = 1)).reshape(-1, 1)
        
        return (l * l_s + r * r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        
        l_part = l_c / l_s
        r_part = r_c / r_s

        l = (1 - np.max(l_part, axis = 1)).reshape(-1, 1)
        r = (1 - np.max(r_part, axis = 1)).reshape(-1, 1)
        
        return (l * l_s + r * r_s) 

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода
        # Сортировка признака x и y по возрастанию x
        # В class_number возвращается количество уникальных классов y
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]
        
        # Что делает этот блок кода?
        # В splitted_sorted_y отрезается min_samples_split минимальных и максимальных значений признака x 
        # В r_border_ids - индексы в splitted_sorted_y, в которых происходит смена класса (со смещением для sprted_y)
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        
        # Что делает этот блок кода?
        # eq_el_count - массив длин последовательностей одинаковых классов
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        
        # one_hot_code - матрица, показывающая переход от одного класса к другому
        # то есть, если слева от i-ой границы был j-й класс, то one_hot_code[i][j] = 1
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        
        # class_increments - матрица, показывающая количество элементов в соответствующем отрезке
        # то есть, если i-ый отрезок имеет n элементов класса j, то class_increments[i][j] = n
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        # добавляем в [0] те элементы, которые были отрезаны
        # (здесь было не sorted_y, а просто y, но мы тогда мы добавим что-то другое)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)
        
        # Что делает этот блок кода?
        # Рассматриваются все возможные разбиения по границе и находятся: 
        # число каждого класса слева и справа от этой границы (в матрицах l_class_count и r_class_count)
        # и размеры левого и правого классов (l_sizes и r_sizes)
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # Вычисляется мера неопределенности для всех вохможных разбиений
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        # Выбирается оптимальное разбиение с наименьшей мерой 
        idx = np.argmin(gs)
    
        # Что делает этот блок кода?
        # Номер элемента, с которого начинается правый класс
        left_el_id = l_sizes[idx][0]
        # Возвращаются: gs[idx] - наименьшая мера неопределенности
        # значение признака, соответствующее разделению (берется середина между значениями граничных элементов) )
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0
   
    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        
        if (len(y) == 0):
            return
        
        probabilities = np.bincount(y) / len(y)
        
        # вычеркнем случаи, когда нужно сразу создавать лист
        if ((depth == self.max_depth) 
            or (x.shape[0] <= self.min_samples_split)
            or ((np.max(np.bincount(y) / y.shape[0] * 1.0) >= self.sufficient_share).any())
            or (np.unique(y).shape[0] == 1)):
                
            self.tree[node_id] = [self.LEAF_TYPE, np.argmax(probabilities), probabilities]
            return
        
        # gs - массив с возможными мерами неопределенности
        gs = np.empty(x.shape[1])
        # thresholds - набор порогов
        thresholds = np.empty(x.shape[1])
        
        for i in self.get_feature_ids(x.shape[1]):
            gs[i], thresholds[i] = self.__find_threshold(x[:, i], y) 
        
        # выбираем оптимальный признак и разбиение
        gs_min = np.argmin(gs)
        optimal_threshold = thresholds[gs_min]
        
        # выполняем разбиение
        x_left, x_right, y_left, y_right = self.__div_samples(x, y, gs_min, optimal_threshold)
        
        # если оба получившихся разбиения не пусты, то создаем два потомка
        if (x_left.shape[0] != 0 and x_right.shape[0] != 0):
            self.__fit_node(x_left, y_left, 2 * node_id + 1, depth + 1)
            self.__fit_node(x_right, y_right, 2 * node_id + 2, depth + 1)
            self.tree[node_id] = [self.__class__.NON_LEAF_TYPE, gs_min, optimal_threshold]
        # иначе это лист
        else: 
            probabilities = np.bincount(y) / len(y)
            self.tree[node_id] = [self.LEAF_TYPE, np.argmax(probabilities), probabilities]

        return
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [52]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [53]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

In [54]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [55]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

0.428357124329
0.982491970062


## Проверка качества работы

In [56]:
gkf = KFold(n_splits=5, shuffle=True)

In [57]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.930780743328
0.931362767107
0.929200964497
0.931944790887
0.930154242714


In [58]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.891244699426
0.893739087054
0.89190986946
0.889124469943
0.89124017794
